<a href="https://colab.research.google.com/github/EiDa21/Fraud-Detection/blob/main/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix


# print(pd.__version__)
# import os
# import pickle


# data = r'C:\Users\HP\Desktop\FRAUD DETECTION\data'

# pickle_files = [f for f in os.listdir(data) if f.endswith('.pkl')]

# the_dataset =[]

# for files in pickle_files:
#     file_path = os.path.join(data, files)
#     with open(file_path, 'rb') as f:
#         df_temp = pickle.load(f)
#         the_dataset.append(df_temp)

# df= pd.concat(the_dataset, ignore_index=True)
# print(df)

# df.to_csv('Fraud_data.csv', index=False)

# df.head(5)
# df.shape
# df.info()
# df.columns
from google.colab import files
uploaded = files.upload()

Saving Fraud_data.csv to Fraud_data (1).csv


In [2]:

df = pd.read_csv('Fraud_data.csv')
print(df.head(5))

print(df.shape)
print(df.info)
print(df.columns)

   TRANSACTION_ID          TX_DATETIME  CUSTOMER_ID  TERMINAL_ID  TX_AMOUNT  \
0               0  2018-04-01 00:00:31          596         3156      57.16   
1               1  2018-04-01 00:02:10         4961         3412      81.51   
2               2  2018-04-01 00:07:56            2         1365     146.00   
3               3  2018-04-01 00:09:29         4128         8737      64.49   
4               4  2018-04-01 00:10:34          927         9906      50.99   

   TX_TIME_SECONDS  TX_TIME_DAYS  TX_FRAUD  TX_FRAUD_SCENARIO  
0               31             0         0                  0  
1              130             0         0                  0  
2              476             0         0                  0  
3              569             0         0                  0  
4              634             0         0                  0  
(1754155, 9)
<bound method DataFrame.info of          TRANSACTION_ID          TX_DATETIME  CUSTOMER_ID  TERMINAL_ID  \
0                     

In [3]:
# LETS GET SOME DATE FEATURES FOR PATTERN DETECTION
df['TX_DATETIME'] = pd.to_datetime(df["TX_DATETIME"], errors= 'coerce')
df['TX_DAY'] = df['TX_DATETIME'].dt.date
df['TX_HOUR'] = df['TX_DATETIME'].dt.hour
df['TX_DAY_OF_WEEK'] = df['TX_DATETIME'].dt.dayofweek

print(df.head(35))
# Average transaction per customer in the past 7days
df = df.sort_values(by=['CUSTOMER_ID', 'TX_DATETIME'])
customer_ave =(
    df.groupby('CUSTOMER_ID')
    .rolling('7D', on= 'TX_DATETIME')['TX_AMOUNT']
    .mean().
    reset_index().
    rename(columns={'TX_AMOUNT':'CUSTOMER_AVG_AMOUNT'})
)
df = df.merge(customer_ave, on=['CUSTOMER_ID','TX_DATETIME'],how='left')

# Count of transactions per terminal in the past 7days
df = df.sort_values(by=['TERMINAL_ID', 'TX_DATETIME'])
terminal_count =(
    df.groupby('TERMINAL_ID')
    .rolling('7D', on= 'TX_DATETIME')['TX_AMOUNT']
    .count()
    .reset_index()
    .rename(columns={'TX_AMOUNT': 'TERMINAL_TX_COUNT'})
    )
df = df.merge(terminal_count, on=['TERMINAL_ID', 'TX_DATETIME'], how='left')
dataset = df
# print(dataset.shape)
# dataset.columns

    TRANSACTION_ID         TX_DATETIME  CUSTOMER_ID  TERMINAL_ID  TX_AMOUNT  \
0                0 2018-04-01 00:00:31          596         3156      57.16   
1                1 2018-04-01 00:02:10         4961         3412      81.51   
2                2 2018-04-01 00:07:56            2         1365     146.00   
3                3 2018-04-01 00:09:29         4128         8737      64.49   
4                4 2018-04-01 00:10:34          927         9906      50.99   
5                5 2018-04-01 00:10:45          568         8803      44.71   
6                6 2018-04-01 00:11:30         2803         5490      96.03   
7                7 2018-04-01 00:11:44         4684         2486      24.36   
8                8 2018-04-01 00:11:53         4128         8354      26.34   
9                9 2018-04-01 00:13:44          541         6212      59.07   
10              10 2018-04-01 00:16:59         4554         2198      58.06   
11              11 2018-04-01 00:17:44         2000 

In [4]:
useful_features = ['TX_AMOUNT', 'TX_DAY_OF_WEEK', 'TX_HOUR', 'CUSTOMER_AVG_AMOUNT','TERMINAL_TX_COUNT']
X = dataset[useful_features]
y= dataset['TX_FRAUD']

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42, test_size=0.2, stratify=y)


In [5]:
model = RandomForestClassifier(random_state=42, n_estimators=100)
model.fit(X_train, y_train)
prediction = model.predict(X_test)

print(classification_report(y_test, prediction))
print('ROC-AUC:', roc_auc_score(y_test, prediction))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00    347934
           1       0.98      0.24      0.39      2937

    accuracy                           0.99    350871
   macro avg       0.99      0.62      0.69    350871
weighted avg       0.99      0.99      0.99    350871

ROC-AUC: 0.6213636812513953
